## Day 28 Lecture 2 Assignment

In this assignment, we will learn about overfitting and regularization. We will use the king county housing dataset loaded below and analyze the regression from this dataset.

In [15]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge

import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
king_county = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/kc_house_data.csv')

In [4]:
king_county.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Perform the same transformations in the previous assignment to meet model assumptions:
1. Transform price and sqft_living by applying the natual log
2. Remove all columns except: log price, bedrooms, bathrooms, log_sqft_living, floors, waterfront
3. Split the data into train and test subsets. 20% of the data should be in the test subset

In [5]:
king_county["log_sqft"] = np.log(king_county["sqft_living"])
king_county = king_county[["price","bedrooms","bathrooms","log_sqft","floors","waterfront"]].drop(index=15870)
king_county["price"] = np.log(king_county["price"])

In [6]:
X = king_county.drop(columns='price')
y = king_county['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36)

Apply a ridge regression model with lambda=50 to the data and evaluate by looking at r squared for test and train

In [10]:
lm = Ridge(alpha=50, random_state=36)
lm.fit(X_train, y_train)


Ridge(alpha=50, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=36, solver='auto', tol=0.001)

In [13]:
display(
    lm.score(X_train, y_train),
    lm.score(X_test, y_test)
)

0.4811098485047658

0.49261971615951056

Perform a grid search for the following values of alpha: 1, 5, 10, 100, 500, 1000, 10000 to find the most optimal ridge regression model.

In [22]:
grid = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 100, 500, 1000, 10000]}
newlm = GridSearchCV(Ridge(), grid, verbose=1)
newlm.fit(X_train, y_train)

bestmodel = newlm.best_estimator_

display(
    bestmodel,
    bestmodel.score(X_train, y_train),
    bestmodel.score(X_test, y_test)
)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    0.2s finished


Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

0.4823638741261336

0.4937510038107814